# The Iris Dataset with Eager Execution



## Setup program

### Install the latest version of TensorFlow



In [0]:
!pip install --upgrade tensorflow

### Configure imports and eager execution



In [0]:
from __future__ import absolute_import, division, print_function

import os
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.contrib.eager as tfe

tf.enable_eager_execution()

print("TensorFlow version: {}".format(tf.VERSION))
print("Eager execution: {}".format(tf.executing_eagerly()))

tf.logging.set_verbosity(tf.logging.ERROR)

## Import and parse the training dataset


### Download the dataset



In [0]:
training_file_url = "http://download.tensorflow.org/data/iris_training.csv"
training_file_path = tf.keras.utils.get_file(
    fname=os.path.basename(training_file_url),
    origin=training_file_url)

print("Location of the training data file: {}".format(training_file_path))

### Inspect the data



In [0]:
!head -n5 {training_file_path}

### Parse the dataset


In [0]:
def parse_csv_fn(line):
  example_defaults = [[0.], [0.], [0.], [0.], [0]]
  parsed_line = tf.decode_csv(line, example_defaults)
  features = tf.reshape(parsed_line[:-1], shape=(4,))
  target = tf.reshape(parsed_line[-1], shape=())
  return features, target

### Create the training tf.data.Dataset



In [0]:
training_dataset = tf.data.TextLineDataset(training_file_path).skip(1).map(parse_csv_fn).shuffle(buffer_size=1000).batch(32)

features, target = iter(training_dataset).next()
print("Features:", features[0])
print("Target:", target[0])

## Select the type of model



### Create a model using Keras



In [0]:
model = tf.keras.Sequential([
  tf.keras.layers.Dense(10, activation="relu", input_shape=(4,)),
  tf.keras.layers.Dense(10, activation="relu"),
  tf.keras.layers.Dense(3)])

## Train the model



### Define the loss and gradient function



In [0]:
def loss_fn(model, features, y):
  return tf.losses.sparse_softmax_cross_entropy(labels=y, logits=model(features))

def gradients_fn(model, features, targets):
  with tf.GradientTape() as tape:
    loss = loss_fn(model, features, targets)
  return tape.gradient(loss, model.variables)

### Create an optimizer



In [0]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)

### Training loop



In [0]:
training_mean_loss = []
training_accuracy = []

num_epochs = 201

for epoch in range(num_epochs):
  epoch_mean_loss = tfe.metrics.Mean()
  epoch_accuracy = tfe.metrics.Accuracy()

  for x, y in training_dataset:
    gradients = gradients_fn(model, x, y)

    optimizer.apply_gradients(
      zip(gradients, model.variables),
      global_step=tf.train.get_or_create_global_step()
    )

    epoch_mean_loss(loss_fn(model, x, y))
    epoch_accuracy(tf.argmax(model(x), axis=1, output_type=tf.int32), y)
  
  training_mean_loss.append(epoch_mean_loss.result())
  training_accuracy.append(epoch_accuracy.result())
  
  if epoch % 10 == 0:
    print("Epoch {:03d}: loss: {:.3f}, accuracy: {:.3%}".format(
      epoch,
      epoch_mean_loss.result(),
      epoch_accuracy.result()))

### Visualize the loss function over time

In [0]:
fig, axes = plt.subplots(2, sharex=True, figsize=(12, 8))
fig.suptitle('Training Metrics')

axes[0].set_ylabel("Loss", fontsize=14)
axes[0].plot(training_mean_loss)

axes[1].set_ylabel("Accuracy", fontsize=14)
axes[1].set_xlabel("Epoch", fontsize=14)
axes[1].plot(training_accuracy)

plt.show()

## Evaluate the model's effectiveness



### Setup the test dataset



In [0]:
test_file_url = "http://download.tensorflow.org/data/iris_test.csv"
test_file_path = tf.keras.utils.get_file(
    fname=os.path.basename(test_file_url),
    origin=test_file_url)

test_dataset = tf.data.TextLineDataset(test_file_path).skip(1).map(parse_csv_fn).shuffle(1000).batch(32)

### Evaluate the model on the test dataset



In [0]:
test_accuracy = tfe.metrics.Accuracy()

for (x, y) in test_dataset:
  prediction = tf.argmax(model(x), axis=1, output_type=tf.int32)
  test_accuracy(prediction, y)

print("Test set accuracy: {:.3%}".format(test_accuracy.result()))

## Use the trained model to make predictions


In [0]:
class_names = ['Iris Setosa', 'Iris Versicolor', 'Iris Virginica']

predict_dataset = tf.convert_to_tensor([
    [5.1, 3.3, 1.7, 0.5,],
    [5.9, 3.0, 4.2, 1.5,],
    [6.9, 3.1, 5.4, 2.1]
])

predictions = model(predict_dataset)

for i, logits in enumerate(predictions):
    class_idx = tf.argmax(logits).numpy()
    class_name = class_names[class_idx]
    print("Example {} prediction: {}".format(i, class_name))